In [17]:
%%writefile datautils.py
# version1
import os
from urllib.request import urlopen

def download(url):
    """Download a file and save it in the current folder.
    Return the name of the downloaded file."""
    # Get the filename.
    file = os.path.basename(url)
    # fix the bug, by specifying a fixed filename if the URL
    if not file:
        file = 'downloaded'
    # Download the file unless it already exists.
    if not os.path.exists(file):
        with open(file, 'w') as f:
            f.write(urlopen(url).read())
    return file

Overwriting datautils.py


In [19]:
%%writefile test_datautils.py
from urllib.request import (HTTPHandler, install_opener, build_opener, addinfourl)
import os
import shutil
import tempfile
from io import StringIO
from datautils import download

TEST_FOLDER = tempfile.mkdtemp()
ORIGINAL_FOLDER = os.getcwd()

class TestHTTPHandler(HTTPHandler):
    """Mock HTTP handler."""
    def http_open(self, req):
        resp = addinfourl(StringIO('test'), '', req.get_full_url(), 200)
        resp.msg = 'OK'
        return resp

def setup():
    """install the mock HTTP handler for unit tests."""
    install_opener(build_opener(TestHTTPHandler))
    os.chdir(TEST_FOLDER)
    
def teardown():
    """Restore the normal HTTP handler."""
    install_opener(build_opener(HTTPHandler))
    # go back to the original folder.
    os.chdir(ORIGINAL_FOLDER)
    # delete the test folder.
    shutil.rmtree(TEST_FOLDER)


def test_download1():
    file = download("http://example.com/file.txt")
    # check that the file has been downloaded.
    assert os.path.exists(file)
    # check that the file contains the contents of the remote file
    with open(file, 'r') as f:
        contents = f.read()
    print(contents)
    assert contents == 'test'

Overwriting test_datautils.py


In [12]:
!nosetests

.
----------------------------------------------------------------------
Ran 1 test in 0.011s

OK


In [20]:
%%writefile test_datautils.py -a

def test_download2():
    file = download("http://example.com/")
    assert os.path.exists(file)

Appending to test_datautils.py


In [21]:
!nosetests

..
----------------------------------------------------------------------
Ran 2 tests in 0.014s

OK
